In [1]:
import aiohttp
import lzma
import io

from datetime import datetime
from os.path import expanduser
from pathlib import Path
from typing import NamedTuple

import pandas as pd
import netaddr

from rpki_analysis.delegated_stats import read_delegated_extended_stats, StatsCombinedAllocations, RirLookup
from rpki_analysis.routinator import read_jsonext

In [2]:
df = await read_jsonext("https://rpki-validator.ripe.net/jsonext")

In [3]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://ftp.ripe.net/pub/stats/ripencc/nro-stats/latest/nro-delegated-stats') as resp:
        df_delext_stats = read_delegated_extended_stats(io.StringIO(await resp.text()))

        rir_lookup = RirLookup(df_delext_stats)
        lookup = StatsCombinedAllocations(df_delext_stats[df_delext_stats.status == 'assigned'])

In [4]:
misses = list()

class PrefixDetails(NamedTuple):
    rir: str
    opaque_id: str
    covering_rir: str

def maybe_lookup(row: pd.DataFrame) -> PrefixDetails:
    containing_rir = rir_lookup.get(row.prefix)
    try:
        entry = lookup[row.prefix]
        return PrefixDetails(entry.rir, entry.opaque_id, containing_rir)
    except KeyError:
        misses.append(row)
        children = list(lookup.children(row.prefix))
        if children:
            child_resources = netaddr.IPSet([c.resource for c in children])
            child_opaque_ids = set(map(lambda r: r.opaque_id, children))
            child_rirs = set(map(lambda r: r.rir, children))
            
            if child_resources.issuperset(netaddr.IPSet([row.prefix])):
                return PrefixDetails(f"{'-'.join(child_rirs)}-multi-opaque-id", "_".join(child_opaque_ids), containing_rir)
                
            return PrefixDetails(f"unknown-more-specific-allocs-{"-".join(child_rirs)}", "_".join(child_opaque_ids), containing_rir)

df[['rir', 'opaque_id', 'containing_rir']] = df.apply(maybe_lookup, axis=1, result_type='expand')
df['publication_point'] = df.uri.apply(lambda uri: "/".join(uri.split("/")[:-1]))
df_unmatched = pd.DataFrame.from_records(misses)

In [5]:
for prefix in set(df[df.containing_rir.isnull()].prefix):
    display(rir_lookup[prefix], prefix)

'apnic'

'103.181.75.0/24'

'arin'

'149.112.106.0/24'

'ripencc'

...

'2001:df0:6ac0::/48'

'apnic'

'160.25.71.0/24'

'apnic'

'160.25.70.0/24'

In [6]:
df_unmatched = df[df.containing_rir == None]

In [7]:
df_delext_stats.status.unique()

['reserved', 'assigned', 'available']
Categories (3, object): ['assigned', 'available', 'reserved']

In [8]:
display(df_unmatched.prefix.unique())
display(df.keys())

array([], dtype=object)

Index(['type', 'uri', 'tal', 'not_before', 'not_after', 'chain_not_before',
       'chain_not_after', 'asn', 'prefix', 'max_length', 'rir', 'opaque_id',
       'containing_rir', 'publication_point'],
      dtype='object')

# Check the tal against the RIR according to delegated extended statistics.

The code below detects some mis-alignment. Many of these will becaused by registration artifacts (two continous registrations controlled by one entity, that are under two IDs in the delegated stats.

In [9]:
#rir_lookup['43.236.0.0/16']
rir_lookup.get('91.207.19.0/24')

'ripencc'

In [10]:
home = Path(expanduser("~"))
now = datetime.now()

df_mismatch = df.loc[df.tal.str.replace("ripe", "ripencc") != df.rir, ['uri', 'tal', 'asn', 'prefix', 'max_length', 'rir', 'opaque_id', 'containing_rir']]
df_mismatch.to_excel(home / f"Desktop/{now.strftime('%Y%m%d')}-results-results.xlsx")
display(df_mismatch)

,uri,tal,asn,prefix,max_length,rir,opaque_id,containing_rir
19382,rsync://rpki.arin.net/repository/arin-rpki-ta/...,arin,401131,23.183.120.0/24,24,None,None,None
26972,rsync://repo-rpki.idnic.net/repo/943e2e67-a171...,apnic,24532,27.111.32.0/19,24,apnic-multi-opaque-id,A92C2F3F_A9227744,apnic
49999,rsync://rpki.cnnic.cn/rpki/A9162E3D0000/890/8b...,apnic,63567,43.226.96.0/20,20,apnic-multi-opaque-id,A92AA1A8_A92E0034_A92681D6_A9246BCD,apnic
50292,rsync://rpki.cnnic.cn/rpki/A9162E3D0000/137/BP...,apnic,24426,43.239.32.0/20,20,apnic-multi-opaque-id,A92EA8D8_A92AE769_A92584A2_A92B7C8D,apnic
50938,rsync://rpki.cnnic.cn/rpki/A9162E3D0000/890/8b...,apnic,63567,43.248.80.0/20,20,apnic-multi-opaque-id,A9238CAA_A929BCD9_A927D326_A9296EFE,apnic
208489,rsync://rpki.apnic.net/member_repository/A918E...,apnic,45769,103.26.232.0/22,24,apnic-multi-opaque-id,A92927D9_A9230910,apnic
208910,rsync://rpki.cnnic.cn/rpki/A9162E3D0000/137/BP...,apnic,24426,103.35.32.0/20,20,apnic-multi-opaque-id,A92EA8D8_A92AE769_A92584A2_A92B7C8D,apnic
215402,rsync://rpki.apnic.net/member_repository/A918E...,apnic,137144,103.106.102.0/23,24,apnic-multi-opaque-id,A9276BC5_A92467C6,apnic
219635,rsync://repo-rpki.idnic.net/repo/IDNIC-ID/2/AS...,apnic,150980,103.138.166.0/23,24,apnic-multi-opaque-id,A92840D5_A92532B9,apnic
226902,rsync://rpki.apnic.net/member_repository/A9183...,apnic,149501,103.181.74.0/24,24,None,None,None


In [11]:
df[df.containing_rir.isnull()]

,type,uri,tal,not_before,not_after,chain_not_before,chain_not_after,asn,prefix,max_length,rir,opaque_id,containing_rir,publication_point
19382,roa,rsync://rpki.arin.net/repository/arin-rpki-ta/...,arin,2024-07-04T01:34:05Z,2024-10-02T01:34:05Z,2024-07-04T01:34:05Z,2024-10-02T01:34:05Z,401131,23.183.120.0/24,24,None,None,None,rsync://rpki.arin.net/repository/arin-rpki-ta/...
226902,roa,rsync://rpki.apnic.net/member_repository/A9183...,apnic,2024-07-03T09:23:42Z,2025-05-01T00:00:00Z,2024-07-03T09:23:42Z,2024-09-06T09:01:43Z,149501,103.181.74.0/24,24,None,None,None,rsync://rpki.apnic.net/member_repository/A9183...
226903,roa,rsync://rpki.apnic.net/member_repository/A9183...,apnic,2024-07-03T09:23:42Z,2025-05-01T00:00:00Z,2024-07-03T09:23:42Z,2024-09-06T09:01:43Z,149501,103.181.75.0/24,24,None,None,None,rsync://rpki.apnic.net/member_repository/A9183...
226904,roa,rsync://rpki.apnic.net/member_repository/A9183...,apnic,2024-07-03T09:23:42Z,2025-05-01T00:00:00Z,2024-07-03T09:23:42Z,2024-09-06T09:01:43Z,149501,103.181.74.0/23,23,None,None,None,rsync://rpki.apnic.net/member_repository/A9183...
288747,roa,rsync://rpki.arin.net/repository/arin-rpki-ta/...,arin,2024-07-02T13:41:09Z,2024-09-30T13:41:09Z,2024-07-02T13:41:09Z,2024-09-30T13:41:09Z,399794,149.112.106.0/24,24,None,None,None,rsync://rpki.arin.net/repository/arin-rpki-ta/...
305023,roa,rsync://rpki.apnic.net/member_repository/A91B4...,apnic,2024-07-03T03:54:55Z,2025-09-30T00:00:00Z,2024-07-03T03:54:55Z,2024-09-06T09:01:43Z,147040,160.25.70.0/24,24,None,None,None,rsync://rpki.apnic.net/member_repository/A91B4...
305024,roa,rsync://rpki.apnic.net/member_repository/A91B4...,apnic,2024-07-03T03:54:55Z,2025-09-30T00:00:00Z,2024-07-03T03:54:55Z,2024-09-06T09:01:43Z,147040,160.25.71.0/24,24,None,None,None,rsync://rpki.apnic.net/member_repository/A91B4...
403423,roa,rsync://rpki.ripe.net/repository/DEFAULT/22/4a...,ripe,2024-07-03T13:44:18Z,2025-07-01T00:00:00Z,2024-07-03T14:19:34Z,2025-07-01T00:00:00Z,42073,194.39.240.0/24,24,None,None,None,rsync://rpki.ripe.net/repository/DEFAULT/22/4a...
403424,roa,rsync://rpki.ripe.net/repository/DEFAULT/ae/19...,ripe,2024-07-04T06:33:18Z,2025-07-01T00:00:00Z,2024-07-04T06:33:18Z,2025-07-01T00:00:00Z,199444,194.39.241.0/24,24,None,None,None,rsync://rpki.ripe.net/repository/DEFAULT/ae/19...
403425,roa,rsync://rpki.ripe.net/repository/DEFAULT/cc/df...,ripe,2024-07-03T14:40:18Z,2025-07-01T00:00:00Z,2024-07-03T14:40:18Z,2025-07-01T00:00:00Z,215686,194.39.242.0/24,24,None,None,None,rsync://rpki.ripe.net/repository/DEFAULT/cc/df...


# Duplicate VRPs

Look at what VRPs are duplicated the most often and how this happens

In [12]:
top_10 = df.groupby(["prefix", "asn", "rir", "max_length"]).count().nlargest(10, ['not_before'])
top_10

,,,,type,uri,tal,not_before,not_after,chain_not_before,chain_not_after,opaque_id,containing_rir,publication_point
prefix,asn,rir,max_length,,,,,,,,,,
102.217.80.0/22,15399,afrinic,22,6,6,6,6,6,6,6,6,6,6
102.215.148.0/22,15399,afrinic,22,5,5,5,5,5,5,5,5,5,5
102.215.160.0/22,15399,afrinic,22,5,5,5,5,5,5,5,5,5,5
102.216.244.0/22,15399,afrinic,22,5,5,5,5,5,5,5,5,5,5
102.216.60.0/22,15399,afrinic,22,5,5,5,5,5,5,5,5,5,5
102.218.28.0/22,15399,afrinic,22,5,5,5,5,5,5,5,5,5,5
102.219.120.0/22,15399,afrinic,22,5,5,5,5,5,5,5,5,5,5
102.219.216.0/22,15399,afrinic,22,5,5,5,5,5,5,5,5,5,5
102.221.20.0/22,15399,afrinic,22,5,5,5,5,5,5,5,5,5,5


# Maximum number of prefixes per ROA
Recall that a ROA has a single AS by definition: The grouping by AS is only to clarify what AS-es have this number

In [13]:
df.groupby(["uri", "asn"]).count().nlargest(10, 'rir')

,,type,tal,not_before,not_after,chain_not_before,chain_not_after,prefix,max_length,rir,opaque_id,containing_rir,publication_point
uri,asn,,,,,,,,,,,,
rsync://rpki.ripe.net/repository/DEFAULT/e2/88fdd1-8b7b-4101-ac66-b590957d9415/1/ii3FV5gnk822mL2XgC_hs2hhKbI.roa,39891,6548,6548,6548,6548,6548,6548,6548,6548,6548,6548,6548,6548
rsync://rpki.apnic.net/member_repository/A91DFB70/2983647C838F11E586FC5812C4F9AE02/9B22F928BFE311EE949C2918C4F9AE02.roa,9299,4266,4266,4266,4266,4266,4266,4266,4266,4266,4266,4266,4266
rsync://rpki.ripe.net/repository/DEFAULT/34/41c321-347e-4c3e-affd-e2b527728e96/1/F8-dcpSK7eR4Q3eZ7DjNFD0hoS0.roa,8551,4127,4127,4127,4127,4127,4127,4127,4127,4127,4127,4127,4127
rsync://rpki.ripe.net/repository/DEFAULT/2f/4b7ef8-8643-4dbb-92aa-0740115b5a52/1/HLyZ7EIByAmhVb-ajQtU8qwlGa4.roa,5416,3933,3933,3933,3933,3933,3933,3933,3933,3933,3933,3933,3933
rsync://rpki.apnic.net/member_repository/A9197CED/83F56DAC1D8411E2910689DA08B02CD2/F99AB520FBC211EEBB321A7DC4F9AE02.roa,9829,3816,3816,3816,3816,3816,3816,3816,3816,3816,3816,3816,3816
rsync://rpki.ripe.net/repository/DEFAULT/79/c22c08-56a7-42d4-bea5-877dda192379/1/5Fhawx5nGuF8_wJjkWopRYaOWfA.roa,54113,2926,2926,2926,2926,2926,2926,2926,2926,2926,2926,2926,2926
rsync://rpki.ripe.net/repository/DEFAULT/fd/9a1d44-8609-4e5e-ba94-5a86c2757c1e/1/RhX4WybtJ7u8HnGwVixXAImSoaM.roa,12389,2876,2876,2876,2876,2876,2876,2876,2876,2876,2876,2876,2876
rsync://rpki.ripe.net/repository/DEFAULT/31/0762da-d66f-4dcb-9c7f-802bb51a1bed/1/UMs8uIBP7mybEF6K3AYBQntR1M8.roa,50710,2719,2719,2719,2719,2719,2719,2719,2719,2719,2719,2719,2719
rsync://rpki.ripe.net/repository/DEFAULT/31/0762da-d66f-4dcb-9c7f-802bb51a1bed/1/UWNAJnUwK24DJTb0itLIyRvDuVo.roa,203214,2719,2719,2719,2719,2719,2719,2719,2719,2719,2719,2719,2719


# Analysis by publication point:
Wat is the maximum number of ROAs, total VRPs per publication point (~= certificate for most CAs)

```
$ rsync rsync://rpki.arin.net/repository/arin-rpki-ta/5e4a23ea-e80a-403e-b08c-2171da2157d3/2a246947-2d62-4a6c-ba05-87187f0099b2/4e95a28e-27fe-479a-b086-2cc9809d54f6/ | wc -l
20729
```

In [14]:
df.groupby(['publication_point']).count().nlargest(10, ['uri'])

,type,uri,tal,not_before,not_after,chain_not_before,chain_not_after,asn,prefix,max_length,rir,opaque_id,containing_rir
publication_point,,,,,,,,,,,,,
rsync://rpki.arin.net/repository/arin-rpki-ta/5e4a23ea-e80a-403e-b08c-2171da2157d3/2a246947-2d62-4a6c-ba05-87187f0099b2/4e95a28e-27fe-479a-b086-2cc9809d54f6,36476,36476,36476,36476,36476,36476,36476,36476,36476,36476,36476,36476,36476
rsync://rpki.apnic.net/member_repository/A918EDB2/2A43E30C70E911E2B36D4B6B2979BB20,13584,13584,13584,13584,13584,13584,13584,13584,13584,13584,13584,13584,13584
rsync://rpki.arin.net/repository/arin-rpki-ta/5e4a23ea-e80a-403e-b08c-2171da2157d3/f60c9f32-a87c-4339-a2f3-6299a3b02e29/5d9c9ef5-b960-412d-af5e-2df18c21f809,9398,9398,9398,9398,9398,9398,9398,9398,9398,9398,9398,9398,9398
rsync://rpki.ripe.net/repository/DEFAULT/31/0762da-d66f-4dcb-9c7f-802bb51a1bed/1,9162,9162,9162,9162,9162,9162,9162,9162,9162,9162,9162,9162,9162
rsync://rpki.ripe.net/repository/DEFAULT/e2/88fdd1-8b7b-4101-ac66-b590957d9415/1,7923,7923,7923,7923,7923,7923,7923,7923,7923,7923,7923,7923,7923
rsync://rpki.arin.net/repository/arin-rpki-ta/5e4a23ea-e80a-403e-b08c-2171da2157d3/f60c9f32-a87c-4339-a2f3-6299a3b02e29/74939129-73e4-4131-8dea-7d0466eca8f3,6060,6060,6060,6060,6060,6060,6060,6060,6060,6060,6060,6060,6060
rsync://rpki.arin.net/repository/arin-rpki-ta/5e4a23ea-e80a-403e-b08c-2171da2157d3/521eb33f-9672-4cd9-acce-137227e971ac/90dcf2c0-6606-4bd9-b445-77b183ef2ff4,6035,6035,6035,6035,6035,6035,6035,6035,6035,6035,6035,6035,6035
rsync://rpki.arin.net/repository/arin-rpki-ta/5e4a23ea-e80a-403e-b08c-2171da2157d3/746e0111-fafb-430f-b778-d204cfcd99a8/11ee0436-2341-4fcc-9197-d4dc6cce847f,4734,4734,4734,4734,4734,4734,4734,4734,4734,4734,4734,4734,4734
rsync://rpki.arin.net/repository/arin-rpki-ta/5e4a23ea-e80a-403e-b08c-2171da2157d3/2a246947-2d62-4a6c-ba05-87187f0099b2/2bc601f7-daff-4d8b-8556-b61503268309,4355,4355,4355,4355,4355,4355,4355,4355,4355,4355,4355,4355,4355


The total number of files per publicationpoint:

In [15]:
df.drop_duplicates(['publication_point', 'uri']).groupby(['publication_point']).count().nlargest(10, ['uri'])

,type,uri,tal,not_before,not_after,chain_not_before,chain_not_after,asn,prefix,max_length,rir,opaque_id,containing_rir
publication_point,,,,,,,,,,,,,
rsync://rpki.arin.net/repository/arin-rpki-ta/5e4a23ea-e80a-403e-b08c-2171da2157d3/2a246947-2d62-4a6c-ba05-87187f0099b2/4e95a28e-27fe-479a-b086-2cc9809d54f6,36476,36476,36476,36476,36476,36476,36476,36476,36476,36476,36476,36476,36476
rsync://rpki.arin.net/repository/arin-rpki-ta/5e4a23ea-e80a-403e-b08c-2171da2157d3/f60c9f32-a87c-4339-a2f3-6299a3b02e29/5d9c9ef5-b960-412d-af5e-2df18c21f809,9075,9075,9075,9075,9075,9075,9075,9075,9075,9075,9075,9075,9075
rsync://rpki.arin.net/repository/arin-rpki-ta/5e4a23ea-e80a-403e-b08c-2171da2157d3/f60c9f32-a87c-4339-a2f3-6299a3b02e29/74939129-73e4-4131-8dea-7d0466eca8f3,6060,6060,6060,6060,6060,6060,6060,6060,6060,6060,6060,6060,6060
rsync://rpki.arin.net/repository/arin-rpki-ta/5e4a23ea-e80a-403e-b08c-2171da2157d3/521eb33f-9672-4cd9-acce-137227e971ac/90dcf2c0-6606-4bd9-b445-77b183ef2ff4,6035,6035,6035,6035,6035,6035,6035,6035,6035,6035,6035,6035,6035
rsync://rpki.arin.net/repository/arin-rpki-ta/5e4a23ea-e80a-403e-b08c-2171da2157d3/746e0111-fafb-430f-b778-d204cfcd99a8/11ee0436-2341-4fcc-9197-d4dc6cce847f,4734,4734,4734,4734,4734,4734,4734,4734,4734,4734,4734,4734,4734
rsync://rpki.arin.net/repository/arin-rpki-ta/5e4a23ea-e80a-403e-b08c-2171da2157d3/2a246947-2d62-4a6c-ba05-87187f0099b2/2bc601f7-daff-4d8b-8556-b61503268309,4354,4354,4354,4354,4354,4354,4354,4354,4354,4354,4354,4354,4354
rsync://rpki-rsync.us-east-2.amazonaws.com/volume/20aa329b-fc52-4c61-bf53-09725c042942,4214,4214,4214,4214,4214,4214,4214,4214,4214,4214,4214,4214,4214
rsync://rpki.arin.net/repository/arin-rpki-ta/5e4a23ea-e80a-403e-b08c-2171da2157d3/746e0111-fafb-430f-b778-d204cfcd99a8/8314711c-e263-425a-adda-3995b7095d9d,3419,3419,3419,3419,3419,3419,3419,3419,3419,3419,3419,3419,3419
rsync://rpki.arin.net/repository/arin-rpki-ta/5e4a23ea-e80a-403e-b08c-2171da2157d3/f60c9f32-a87c-4339-a2f3-6299a3b02e29/73298062-bd8b-4b9f-8613-deb6e9c20878,2668,2668,2668,2668,2668,2668,2668,2668,2668,2668,2668,2668,2668


publication points generally contain one ROA for an AS, let's check

# TODO

Now count prefixes per publication point

# Now let's work on unique VRPs

In [16]:
df = df.reset_index().drop_duplicates(['asn', 'prefix', 'max_length'])

The ROA with the most prefixes:

```python
```

In [17]:
df.groupby(["asn"]).count().nlargest(10, ['index'])

,index,type,uri,tal,not_before,not_after,chain_not_before,chain_not_after,prefix,max_length,rir,opaque_id,containing_rir,publication_point
asn,,,,,,,,,,,,,,
16509,8822,8822,8822,8822,8822,8822,8822,8822,8822,8822,8822,8822,8822,8822
39891,6554,6554,6554,6554,6554,6554,6554,6554,6554,6554,6554,6554,6554,6554
7015,6433,6433,6433,6433,6433,6433,6433,6433,6433,6433,6433,6433,6433,6433
22773,5766,5766,5766,5766,5766,5766,5766,5766,5766,5766,5766,5766,5766,5766
14618,5383,5383,5383,5383,5383,5383,5383,5383,5383,5383,5383,5383,5383,5383
20940,4746,4746,4746,4746,4746,4746,4746,4746,4746,4746,4746,4746,4746,4746
7155,4340,4340,4340,4340,4340,4340,4340,4340,4340,4340,4340,4340,4340,4340
9299,4340,4340,4340,4340,4340,4340,4340,4340,4340,4340,4340,4340,4340,4340
54113,4173,4173,4173,4173,4173,4173,4173,4173,4173,4173,4173,4173,4173,4173


Prefix with most ROAs:

In [18]:
df.groupby(['prefix']).count().nlargest(10, ['roa'])

KeyError: 'roa'